<a href="https://colab.research.google.com/github/Lossophy/BINA-Projekt/blob/main/import%20git%20-%20Kopie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verbindung zu GIT

In [1]:
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git

Cloning into 'BINA-Projekt'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 113 (delta 58), reused 34 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 959.90 KiB | 7.06 MiB/s, done.
Resolving deltas: 100% (58/58), done.


Import librarys

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display
from IPython.display import clear_output
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'notebook'  # oder 'iframe' oder 'plotly_mimetype'


In [4]:
df = pd.read_csv('/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv')

In [4]:
df.head()

,Jahr,Rubrik,Energietraeger,TJ
0,1980,Bruttoverbrauch,Elektrizität,-29450.0
1,1980,Bruttoverbrauch,Erdölprodukte,321250.0
2,1980,Bruttoverbrauch,Fernwärme,0.0
3,1980,Bruttoverbrauch,Gas,36280.0
4,1980,Bruttoverbrauch,Holzenergie,26280.0


In [5]:
von_jahr_widget = widgets.Dropdown(
    options=sorted(df['Jahr'].unique()),
    description='Jahr',
    disabled=False,
)

In [6]:
bis_jahr_widget = widgets.Dropdown(
    options=sorted(df['Jahr'].unique()),
    description='Jahr',
    disabled=False,
)

In [7]:
rubrik_widget = widgets.Dropdown(
    options=sorted(df['Rubrik'].unique()),
    value=df['Rubrik'].unique()[0],
    description='Rubrik:'
)


In [8]:
energietraeger_widget = widgets.Dropdown(
    options=sorted(df['Energietraeger'].unique()),
    value=df['Energietraeger'].unique()[0],
    description='energietraeger:'
)


In [9]:
multi_energietraeger_widget = widgets.SelectMultiple(
    options=sorted(df['Energietraeger'].unique()),
    value=tuple(df['Energietraeger'].unique()),
    description='Energie:'
)

In [10]:
# Interaktive Funktion definieren
def bal_plot_daten(jahr, rubrik, energietraeger):
    gefiltert = df[(df['Jahr'] == jahr) &
                   (df['Rubrik'] == rubrik) &
                   (df['Energietraeger'] == energietraeger)]

    if gefiltert.empty:
        print("Keine Daten für diese Kombination.")
        return

    plt.figure(figsize=(6,4))
    plt.bar([0], gefiltert['TJ'], tick_label=[f"{jahr}"])
    plt.ylabel('TJ')
    plt.title(f"{rubrik} - {energietraeger} ({jahr})")
    plt.show()

In [11]:
# Interaktive Ansicht verbinden
ui = widgets.VBox([von_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(bal_plot_daten, {
    'jahr': von_jahr_widget,
    'rubrik': rubrik_widget,
    'energietraeger': energietraeger_widget
})

display(ui, out)

Output()

In [37]:
# Plot-Funktion
def plot_daten(von_jahr, bis_jahr, rubrik, energietraeger):
    if von_jahr > bis_jahr:
        print("Fehler: 'Von Jahr' darf nicht größer als 'Bis Jahr' sein.")
        return

    gefiltert = df[(df['Jahr'] >= von_jahr) &
                   (df['Jahr'] <= bis_jahr) &
                   (df['Rubrik'] == rubrik) &
                   (df['Energietraeger'] == energietraeger)]

    if gefiltert.empty:
        print("Keine Daten für diese Auswahl.")
        return

    gruppiert = gefiltert.groupby('Jahr')['TJ'].sum().reset_index()

    plt.figure(figsize=(25,10))
    plt.plot(gruppiert['Jahr'].astype(int), gruppiert['TJ'], marker='o')
    plt.xticks(gruppiert['Jahr'].astype(int))
    plt.xlabel('Jahr')
    plt.ylabel('TJ')
    plt.title(f'{rubrik} - {energietraeger} ({von_jahr}–{bis_jahr})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Anzeige
ui = widgets.VBox([von_jahr_widget, bis_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(plot_daten, {
    'von_jahr': von_jahr_widget,
    'bis_jahr': bis_jahr_widget,
    'rubrik': rubrik_widget,
    'energietraeger': energietraeger_widget
})
clear_output()
display(ui, out)


Output()

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import seaborn as sns

# Stil setzen (modern & clean)
sns.set_style("whitegrid")
plt.rcParams.update({
    "figure.figsize": (14, 6),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12
})

jahre = sorted(df['Jahr'].unique())
rubriken = ['(Alle Rubriken)'] + sorted(df['Rubrik'].dropna().unique())

# Funktion mit schönerem Plot
def plot_energie(jahr_von, jahr_bis, rubrik_auswahl):
    daten = df[(df['Jahr'] >= jahr_von) & (df['Jahr'] <= jahr_bis)]

    if rubrik_auswahl != '(Alle Rubriken)':
        daten = daten[daten['Rubrik'] == rubrik_auswahl]

    pivot = daten.pivot_table(index='Jahr', columns='Energietraeger', values='TJ', aggfunc='sum')

    ax = pivot.plot(marker='o', linewidth=2)

    # Stilistische Verbesserungen
    plt.title(f"Energieverbrauch ({rubrik_auswahl}) {jahr_von}–{jahr_bis}", fontsize=20, weight='bold')
    plt.xlabel("Jahr")
    plt.ylabel("Verbrauch in TJ")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(title='Energieträger', loc='upper left', bbox_to_anchor=(1.0, 1.0))
    sns.despine()
    plt.tight_layout()
    plt.show()

# Interaktive Oberfläche
interact(
    plot_energie,
    jahr_von=widgets.IntSlider(min=min(jahre), max=max(jahre), step=1, value=min(jahre), description='Von Jahr'),
    jahr_bis=widgets.IntSlider(min=min(jahre), max=max(jahre), step=1, value=max(jahre), description='Bis Jahr'),
    rubrik_auswahl=widgets.Dropdown(options=rubriken, description='Rubrik'),
)


interactive(children=(IntSlider(value=1980, description='Von Jahr', max=2023, min=1980), IntSlider(value=2023,…

<function __main__.plot_energie(jahr_von, jahr_bis, rubrik_auswahl)>

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import seaborn as sns

# === Plot-Stil setzen ===
sns.set_style("whitegrid")
plt.rcParams.update({
    "figure.figsize": (14, 6),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12
})

jahre = sorted(df['Jahr'].unique())
rubriken = ['(Alle Rubriken)'] + sorted(df['Rubrik'].dropna().unique())

# === Interaktive Plot-Funktion ===
def plot_area(jahr_von, jahr_bis, rubrik_auswahl):
    daten = df[(df['Jahr'] >= jahr_von) & (df['Jahr'] <= jahr_bis)]

    if rubrik_auswahl != '(Alle Rubriken)':
        daten = daten[daten['Rubrik'] == rubrik_auswahl]

    pivot = daten.pivot_table(index='Jahr', columns='Energietraeger', values='TJ', aggfunc='sum')
    pivot.fillna(0, inplace=True)

    # === Area Plot ===
    ax = pivot.plot.area(alpha=0.7)

    plt.title(f"Energieverbrauch ({rubrik_auswahl}) {jahr_von}–{jahr_bis}", fontsize=20, weight='bold')
    plt.xlabel("Jahr")
    plt.ylabel("Verbrauch in TJ")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend(title='Energieträger', loc='upper left', bbox_to_anchor=(1.0, 1.0))
    sns.despine()
    plt.tight_layout()
    plt.show()

# === Interaktive Steuerung ===
interact(
    plot_area,
    jahr_von=widgets.IntSlider(min=min(jahre), max=max(jahre), step=1, value=min(jahre), description='Von Jahr'),
    jahr_bis=widgets.IntSlider(min=min(jahre), max=max(jahre), step=1, value=max(jahre), description='Bis Jahr'),
    rubrik_auswahl=widgets.Dropdown(options=rubriken, description='Rubrik'),
)


interactive(children=(IntSlider(value=1980, description='Von Jahr', max=2023, min=1980), IntSlider(value=2023,…

<function __main__.plot_area(jahr_von, jahr_bis, rubrik_auswahl)>